# Load Dependencies

In [2]:
import fire
import torch
import numpy as np
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, set_seed

from transformers import DataCollatorForSeq2Seq

from tqdm.auto import tqdm
import evaluate

# Helper Functions

In [3]:
id2label = {0:'entailment', 1:'neutral', 2:'contradiction'}
label2id = {'entailment':0, 'neutral':1, 'contradiction':2}
num_labels = len(id2label)
max_target_length = 5

def preprocess_snli_batch(examples):
    premises = examples['premise']
    hypotheses = examples['hypothesis']
    labels = examples['label']

    def generate_input(_premise, _hypothesis):
        return " ".join(["premise:", _premise, "hypothesis:", _hypothesis])

    inputs = [generate_input(premise, hypothesis) for premise, hypothesis in zip(premises, hypotheses)]
    targets = [id2label[label] if (label) in range(num_labels) else "" for label in labels]
    return inputs, targets

def convertlabels2ids(example):
    example['label'] = label2id[example['label']]
    return example

def preprocess_function(examples):
    inputs, targets = preprocess_snli_batch(examples)
    model_inputs = tokenizer(inputs)
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def build_dataset(tokenizer, num_proc=4):
    dataset = load_dataset("snli")
    dataset = dataset.filter(lambda sample: sample['label'] in id2label)
    tokenized_dataset = dataset.map(preprocess_function, batched=True, num_proc=num_proc)
    tokenized_dataset = tokenized_dataset.remove_columns(['premise','hypothesis','label'])
    train_dataset = tokenized_dataset["train"]
    validation_dataset = tokenized_dataset["validation"]
    return train_dataset, validation_dataset

def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    pred_ids = [label2id[p] if p in label2id else -1 for p in decoded_preds]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    label_ids = [label2id[l] if l in label2id else -1 for l in decoded_labels]

    result = metric.compute(predictions=pred_ids, references=label_ids) 
    return result
    
def build_mnli_dataset(tokenizer, num_proc=4):
    dataset = load_dataset("multi_nli")
    dataset = dataset.filter(lambda sample: sample['label'] in id2label)
    tokenized_dataset = dataset.map(preprocess_function, batched=True, num_proc=num_proc)
    tokenized_dataset = tokenized_dataset.remove_columns(['premise','hypothesis','label'])
    train_dataset = tokenized_dataset["train"]
    validation_matched_dataset = tokenized_dataset["validation_matched"]
    validation_mismatched_dataset = tokenized_dataset['validation_mismatched']
    return train_dataset, validation_matched_dataset, validation_mismatched_dataset

# Load Model and Tokenizer

In [4]:
checkpoint = 't5-base'
seed = 42

# Set Seed
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

# Load Model and Tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/varu/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# Build Dataset

In [5]:
train_dataset, validation_matched_dataset, validation_mismatched_dataset = build_mnli_dataset(tokenizer, num_proc=4)
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Filter:   0%|          | 0/392702 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9815 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/392702 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/9815 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/9832 [00:00<?, ? examples/s]

In [6]:
def length_filter(examples):
    input_len = len(examples['input_ids'])
    return (input_len < tokenizer.model_max_length)

train_dataset = train_dataset.filter(length_filter)

Filter:   0%|          | 0/392702 [00:00<?, ? examples/s]

# Train

In [ ]:
batch_size = 32
num_train_epochs = 22
metric_name = 'accuracy'

# Training Args
args = Seq2SeqTrainingArguments(
    "/scratch/vgvinodv_root/vgvinodv0/varu/tmp_t5_dir",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    overwrite_output_dir=True,
    fp16=True,
    #push_to_hub=True,
)

# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_matched_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

# Training
trainer.train() 

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.114000,0.124378,0.864595
2,0.134500,0.136698,0.853999


/home/varu/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


/home/varu/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Evaluate

In [ ]:
results = trainer.evaluate(validation_mismatched_dataset)
print(results['eval_accuracy']*100)

In [ ]:
results = trainer.evaluate(validation_matched_dataset)
print(results['eval_accuracy']*100)

# Old code (ignore)

In [ ]:
batch_size = 32
max_steps =  2**18#262144 # 2^18
metric_name = 'accuracy'

# Training Args
args = Seq2SeqTrainingArguments(
    "tmp_t5_dir",
    evaluation_strategy = "epoch",
    save_strategy = "no",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    max_steps=max_steps,
    predict_with_generate=True,
    #load_best_model_at_end=True,
    #metric_for_best_model=metric_name,
    overwrite_output_dir=True,
    fp16=True,
    #push_to_hub=True,
)

# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_matched_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Training
trainer.train() 